### import package

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules
import matplotlib.pyplot as plt
plt.style.use('default')

### import data dari google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#@title 
file_path = "/content/gdrive/MyDrive/dts_project/dataset/association_product_dataset.csv" #@param {type:'string'}
print(f"Used File Path: {file_path}")

Used File Path: /content/gdrive/MyDrive/dts_project/dataset/association.csv


In [ ]:
data = pd.read_csv(file_path, encoding = 'windows-1252')
data.head()

,customer_name,_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,_11,_12,_13,_14
0,2014-01-03 | Darren Powers,Paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-04 | Phillina Ober,Labels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-05 | Mick Brown,Art,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-06 | Jack O'Briant,Art,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-06 | Lycoris Saunders,Paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### import data dari bigquery

In [ ]:
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

pada proses import data, dilakukan filter data dengan mengambil Profit > 0 (per Order_ID)

In [ ]:
# %%bigquery --project golden-resolver-357006 data
# WITH src AS (
#   SELECT
#     v.*
#   FROM
#     `uprak.super_store_mat` v
#   WHERE 
#     v.Profit > 0
#   ORDER BY
#     1,
#     2,
#     3
# ),
# ready as (
# SELECT
#   concat(order_date," | ",customer_name) customer_name,
#   sub_category value,
#   ROW_NUMBER() OVER(PARTITION BY order_date, customer_name) key
# FROM
#   src
# ORDER BY
#   order_date ASC
# )
# SELECT 
#   customer_name, 
#   MAX(IF(key = 1, value, NULL)) AS `_1`,
#   MAX(IF(key = 2, value, NULL)) AS `_2`,
#   MAX(IF(key = 3, value, NULL)) AS `_3`,
#   MAX(IF(key = 4, value, NULL)) AS `_4`,
#   MAX(IF(key = 5, value, NULL)) AS `_5`,
#   MAX(IF(key = 6, value, NULL)) AS `_6`,
#   MAX(IF(key = 7, value, NULL)) AS `_7`,
#   MAX(IF(key = 8, value, NULL)) AS `_8`,
#   MAX(IF(key = 9, value, NULL)) AS `_9`,
#   MAX(IF(key = 10, value, NULL)) AS `_10`,
#   MAX(IF(key = 11, value, NULL)) AS `_11`,
#   MAX(IF(key = 12, value, NULL)) AS `_12`,
#   MAX(IF(key = 13, value, NULL)) AS `_13`,
#   MAX(IF(key = 14, value, NULL)) AS `_14`,
# FROM ready 
# GROUP BY customer_name
# ORDER BY 1

In [ ]:
# data.head()

### pre-process

In [ ]:
data=data.drop(columns=['customer_name'])

In [ ]:
data.columns=[int(bla.replace("_","")) for bla in data.columns]

In [ ]:
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Labels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Art,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Art,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### visualize

In [ ]:
data.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
count,4393,1948,906,438,196,90,44,21,11,5,3,1,1,1
unique,17,17,17,17,17,16,14,11,8,3,3,1,1,1
top,Paper,Paper,Paper,Paper,Paper,Phones,Paper,Accessories,Paper,Paper,Paper,Accessories,Phones,Chairs
freq,754,321,178,55,31,14,12,5,3,2,1,1,1,1


In [ ]:
# transform to array
transaction = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        transaction.append(data.values[i,j])

transaction = np.array(transaction)

# transform to dataframe
df = pd.DataFrame(transaction, columns=["items"]) 
df["incident_count"] = 1

# remove none
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)

# table for visualize
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

# visualize
df_table.head(10).style.background_gradient(cmap='Blues')

,items,incident_count
0,Paper,1370
1,Binders,910
2,Art,796
3,Furnishings,781
4,Phones,751
5,Accessories,683
6,Storage,661
7,Appliances,399
8,Labels,364
9,Chairs,362


In [ ]:
df_table["all"] = "all" 

fig = px.treemap(df_table.head(30), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(30), hover_data=['items'],
                  color_continuous_scale='Blues',
                  )
fig.show()

In [ ]:
transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])
    
transaction = np.array(transaction)

# check status top 20 items

top20 = df_table["items"].head(20).values
array = []
df_top20_multiple_record_check = pd.DataFrame(columns=top20)

for i in range(0, len(top20)):
    array = []
    for j in range(0,transaction.shape[0]):
        array.append(np.count_nonzero(transaction[j]==top20[i]))
        if len(array) == len(data):
            df_top20_multiple_record_check[top20[i]] = array
        else:
            continue
            

df_top20_multiple_record_check.head(10)

,Paper,Binders,Art,Furnishings,Phones,Accessories,Storage,Appliances,Labels,Chairs,Envelopes,Fasteners,Supplies,Bookcases,Tables,Machines,Copiers
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,0,2,0,0,0,0,1,0,1,0,0,0,0,0
6,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [ ]:
df_top20_multiple_record_check.describe()

,Paper,Binders,Art,Furnishings,Phones,Accessories,Storage,Appliances,Labels,Chairs,Envelopes,Fasteners,Supplies,Bookcases,Tables,Machines,Copiers
count,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000,4393.000000
mean,0.311860,0.207148,0.181197,0.177783,0.170954,0.155475,0.150467,0.090826,0.082859,0.082404,0.057819,0.045527,0.035739,0.026633,0.025950,0.016162,0.015479
std,0.550863,0.474173,0.425659,0.419295,0.411197,0.390813,0.389275,0.299042,0.290185,0.299578,0.238255,0.208481,0.188096,0.162435,0.163245,0.126113,0.123463
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,4.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000


In [ ]:
# top first choice

# transform to array
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,0])

transaction = np.array(transaction)

# transform to dataframe
df_first = pd.DataFrame(transaction, columns=["items"])
df_first["incident_count"] = 1

# table for visualize
indexNames = df_first[df_first['items'] == "nan" ].index
df_first.drop(indexNames , inplace=True)

# visualize
df_table_first = df_first.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_first["super_store"] = "super_store"
df_table_first = df_table_first.truncate(before=-1, after=15) # top 15 1st choice

In [ ]:
fig = go.Figure(data=[go.Bar(x=df_table_first["items"], y=df_table_first["incident_count"],
            hovertext=df_table_first["items"], text=df_table_first["incident_count"], textposition="outside")])

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.65)
fig.update_layout(title_text="pilihan peratama customer", template="plotly_dark")
fig.show()

In [ ]:
# top second choice

# transform to array
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,1])

transaction = np.array(transaction)

# transform to dataframe
df_second = pd.DataFrame(transaction, columns=["items"]) 
df_second["incident_count"] = 1

# table for visualize
indexNames = df_second[df_second['items'] == "nan" ].index
df_second.drop(indexNames , inplace=True)

# visualize
df_table_second = df_second.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_second["super_store"] = "super_store"
df_table_second = df_table_second.truncate(before=-1, after=15) # top 15 2nd choice

In [ ]:
fig = go.Figure(data=[go.Bar(x=df_table_second["items"], y=df_table_second["incident_count"],
            hovertext=df_table_second["items"], text=df_table_second["incident_count"], textposition="outside")])

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.65)
fig.update_layout(title_text="pilihan kedua customer", template="plotly_dark")
fig.show()

In [ ]:
# top third choice
# transform to array
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,2])

transaction = np.array(transaction)

# transform to dataframe
df_third = pd.DataFrame(transaction, columns=["items"]) 
df_third["incident_count"] = 1 

# table for visualize
indexNames = df_third[df_third['items'] == "nan" ].index
df_third.drop(indexNames , inplace=True)

# visualize
df_table_third = df_third.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_third["super_store"] = "super_store"
df_table_third = df_table_third.truncate(before=-1, after=15) # top 15 third choice

In [ ]:
fig = go.Figure(data=[go.Bar(x=df_table_third["items"], y=df_table_third["incident_count"],
            hovertext=df_table_third["items"], text=df_table_third["incident_count"], textposition="outside")])

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.65)
fig.update_layout(title_text="pilihan ketiga customer", template="plotly_dark")
fig.show()

### main-process

In [ ]:
# transform data to array

transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])
    
transaction = np.array(transaction)
transaction

array([['Paper', 'nan', 'nan', ..., 'nan', 'nan', 'nan'],
       ['Labels', 'nan', 'nan', ..., 'nan', 'nan', 'nan'],
       ['Art', 'nan', 'nan', ..., 'nan', 'nan', 'nan'],
       ...,
       ['Appliances', 'nan', 'nan', ..., 'nan', 'nan', 'nan'],
       ['Binders', 'Binders', 'nan', ..., 'nan', 'nan', 'nan'],
       ['Phones', 'Bookcases', 'Binders', ..., 'nan', 'nan', 'nan']],
      dtype='<U11')

In [ ]:
# transform to dataframe
# transaction encoder

te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset

,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables,nan
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4388,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True
4389,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True
4390,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4391,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [ ]:
n=18 # Select Top-n item (columns)
firstn = df_table["items"].head(n).values # Select Top-n item (columns)
dataset = dataset.loc[:,firstn] # Extract Top-n
dataset

,Paper,Binders,Art,Furnishings,Phones,Accessories,Storage,Appliances,Labels,Chairs,Envelopes,Fasteners,Supplies,Bookcases,Tables,Machines,Copiers
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4388,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4389,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4390,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4391,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
# transform to binary encoding

def encode_units(x):
    if x == False:
        return 0 
    if x == True:
        return 1
        
dataset = dataset.applymap(encode_units)
dataset.head(10)

,Paper,Binders,Art,Furnishings,Phones,Accessories,Storage,Appliances,Labels,Chairs,Envelopes,Fasteners,Supplies,Bookcases,Tables,Machines,Copiers
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0
6,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [ ]:
# mencari apriori dari masing-masing itemsets
# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

frequent_itemsets = apriori(dataset, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.271113,(Paper),1
1,0.179832,(Binders),1
2,0.166401,(Art),1
3,0.164125,(Furnishings),1
4,0.157979,(Phones),1
5,0.145459,(Accessories),1
6,0.139085,(Storage),1
7,0.087639,(Appliances),1
8,0.078762,(Labels),1
9,0.077168,(Chairs),1


In [ ]:
# membuat association_rules berdasarkan frequents_item dari hasil apriori
# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
33,(Appliances),(Binders),0.087639,0.179832,0.018894,0.215584,1.198813,0.003133,1.045579,1,1
32,(Binders),(Appliances),0.179832,0.087639,0.018894,0.105063,1.198813,0.003133,1.019469,1,1
69,(Appliances),(Phones),0.087639,0.157979,0.015024,0.171429,1.085138,0.001179,1.016233,1,1
68,(Phones),(Appliances),0.157979,0.087639,0.015024,0.095101,1.085138,0.001179,1.008246,1,1
59,(Appliances),(Furnishings),0.087639,0.164125,0.015479,0.176623,1.076153,0.001095,1.015180,1,1
...,...,...,...,...,...,...,...,...,...,...,...
23,(Art),(Binders),0.166401,0.179832,0.022308,0.134063,0.745492,-0.007616,0.947146,1,1
43,(Accessories),(Art),0.145459,0.166401,0.017073,0.117371,0.705349,-0.007132,0.944450,1,1
42,(Art),(Accessories),0.166401,0.145459,0.017073,0.102599,0.705349,-0.007132,0.952240,1,1
39,(Furnishings),(Art),0.164125,0.166401,0.018666,0.113731,0.683475,-0.008644,0.940571,1,1


In [ ]:
# mengurutkan data berdasar confidence score
rules.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
20,(Fasteners),(Paper),0.045527,0.271113,0.012748,0.280000,1.032779,0.000405,1.012343,1,1
12,(Appliances),(Paper),0.087639,0.271113,0.022991,0.262338,0.967632,-0.000769,0.988104,1,1
17,(Chairs),(Paper),0.077168,0.271113,0.018438,0.238938,0.881322,-0.002483,0.957723,1,1
11,(Storage),(Paper),0.139085,0.271113,0.032552,0.234043,0.863265,-0.005156,0.951602,1,1
18,(Envelopes),(Paper),0.056681,0.271113,0.013203,0.232932,0.859168,-0.002164,0.950224,1,1
...,...,...,...,...,...,...,...,...,...,...,...
62,(Furnishings),(Chairs),0.164125,0.077168,0.010471,0.063800,0.826769,-0.002194,0.985721,1,1
50,(Art),(Chairs),0.166401,0.077168,0.010471,0.062927,0.815459,-0.002370,0.984803,1,1
34,(Binders),(Labels),0.179832,0.078762,0.011154,0.062025,0.787506,-0.003010,0.982157,1,1
19,(Paper),(Envelopes),0.271113,0.056681,0.013203,0.048699,0.859168,-0.002164,0.991609,1,1


berdasar hasil diatas, Paper mendominasi consequents<br>
sehingga kita coba take out Paper dari consequents

In [ ]:
rules[~rules["antecedents"].str.contains("Paper", regex=False) &
      ~rules["consequents"].str.contains("Paper", regex=False)].sort_values("confidence", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
33,(Appliances),(Binders),0.087639,0.179832,0.018894,0.215584,1.198813,0.003133,1.045579,1,1
31,(Storage),(Binders),0.139085,0.179832,0.026861,0.193126,1.073927,0.001849,1.016477,1,1
37,(Chairs),(Binders),0.077168,0.179832,0.013658,0.176991,0.984205,-0.000219,0.996549,1,1
59,(Appliances),(Furnishings),0.087639,0.164125,0.015479,0.176623,1.076153,0.001095,1.015180,1,1
69,(Appliances),(Phones),0.087639,0.157979,0.015024,0.171429,1.085138,0.001179,1.016233,1,1
27,(Phones),(Binders),0.157979,0.179832,0.025723,0.162824,0.905426,-0.002687,0.979685,1,1
53,(Phones),(Furnishings),0.157979,0.164125,0.024812,0.157061,0.956958,-0.001116,0.991620,1,1
25,(Furnishings),(Binders),0.164125,0.179832,0.025723,0.156727,0.871520,-0.003792,0.972601,1,1
41,(Phones),(Art),0.157979,0.166401,0.023902,0.151297,0.909230,-0.002386,0.982203,1,1
52,(Furnishings),(Phones),0.164125,0.157979,0.024812,0.151179,0.956958,-0.001116,0.991989,1,1


berdasar hasil diatas, Binders mendominasi consequents <br>
sehingga kita coba take out Paper & Binders dari consequents

In [ ]:
rules[~rules["antecedents"].str.contains("Paper", regex=False) &
      ~rules["consequents"].str.contains("Paper", regex=False) &
      ~rules["antecedents"].str.contains("Binders", regex=False) &
      ~rules["consequents"].str.contains("Binders", regex=False)].sort_values("confidence", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
59,(Appliances),(Furnishings),0.087639,0.164125,0.015479,0.176623,1.076153,0.001095,1.015180,1,1
69,(Appliances),(Phones),0.087639,0.157979,0.015024,0.171429,1.085138,0.001179,1.016233,1,1
53,(Phones),(Furnishings),0.157979,0.164125,0.024812,0.157061,0.956958,-0.001116,0.991620,1,1
41,(Phones),(Art),0.157979,0.166401,0.023902,0.151297,0.909230,-0.002386,0.982203,1,1
52,(Furnishings),(Phones),0.164125,0.157979,0.024812,0.151179,0.956958,-0.001116,0.991989,1,1
81,(Chairs),(Storage),0.077168,0.139085,0.011382,0.147493,1.060450,0.000649,1.009862,1,1
40,(Art),(Phones),0.166401,0.157979,0.023902,0.143639,0.909230,-0.002386,0.983255,1,1
78,(Labels),(Storage),0.078762,0.139085,0.011154,0.141618,1.018216,0.000200,1.002952,1,1
67,(Storage),(Phones),0.139085,0.157979,0.019577,0.140753,0.890962,-0.002396,0.979953,1,1
71,(Labels),(Phones),0.078762,0.157979,0.010926,0.138728,0.878146,-0.001516,0.977649,1,1


berikut sudah kita dapatkan confidence score dari hasil masing-masing Bundling<br>
<br>
berdasrkan hasil EDA, lebih baik merekomendasikan product bundling dengan	Category Office Supplies (Binders, Art, Labels, Paper, Envelopes, Appliances, Storage, Supplies, Fasteners)<br>
<br>
karena Category Office Supplies memiliki harga yang cenderung murah (cocok untuk Bundling) serta memiliki profit kelas menengah (lebih tinggi dari Category Furniture)

# daftar pustaka
1. https://www.kaggle.com/code/evrenermis/association-rule-based-learning-explained/notebook <br>
2. http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ <br>
3. http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/